In [1]:
from bbox.metrics import iou_3d
from kiss_icp.tools.utils_class import BoundingBox3D
from typing import List
from scipy.spatial.transform import Rotation as R
import numpy as np
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
annotations_and_detections = np.load("../results/annotations_and_detections/004/004001.npy", allow_pickle='TRUE').item()

In [3]:
def compute_translation_error(p0, p1):
    dist = np.linalg.norm(p0 - p1)
    return dist

In [4]:
def compute_orientation_error(anno_R, detected_R):
    if anno_R[0][0] < 0 and anno_R[1][1] < 0:
        anno_R[0][0] = -anno_R[0][0] 
        anno_R[0][1] = -anno_R[0][1] 
        anno_R[1][0] = -anno_R[1][0] 
        anno_R[1][1] = -anno_R[1][1] 
    
    if detected_R[0][0] < 0 and detected_R[1][1] < 0:
        detected_R[0][0] = -detected_R[0][0] 
        detected_R[0][1] = -detected_R[0][1] 
        detected_R[1][0] = -detected_R[1][0] 
        detected_R[1][1] = -detected_R[1][1] 
        
    anno_r = R.from_matrix(anno_R)
    detected_r = R.from_matrix(detected_R)
    anno_euler = anno_r.as_euler('zxy', degrees=True)
    detected_euler = detected_r.as_euler('zxy', degrees=True)
    anno_yaw = anno_euler[0]
    detected_yaw = detected_euler[0]
    diff = np.abs(anno_yaw -  detected_yaw)
    
    # print("detected_R", detected_R)
    # print("anno_R", anno_R)
    
    # print("detected_euler", detected_euler)
    # print("anno_euler", anno_euler)

    # print("detected_yaw", detected_yaw)
    # print("anno_yaw", anno_yaw)
    # print("diff", diff)
    return diff

In [5]:
# print("annotations_and_detections\n", annotations_and_detections)

evaluation = {}
# for key, value in my_dict.items():
# for frame_id in range(len(annotations_and_detections)):
for frame_id in annotations_and_detections:
    frame = annotations_and_detections[frame_id]

    for track_uuid, anno_and_det in frame.items():

        try:
            if evaluation[track_uuid]:
                pass
        except:
            evaluation[track_uuid] = {}
        # if not evaluation[track_uuid]:
        #     evaluation[track_uuid] = {}
            
        evaluation[track_uuid][frame_id] = {}
        detection = anno_and_det[0]
        annotation = anno_and_det[1]

        annotation_bbox = BoundingBox3D(annotation.dst_SE3_object.translation[0], annotation.dst_SE3_object.translation[1],
        annotation.dst_SE3_object.translation[2], annotation.length_m, 
        annotation.width_m, annotation.height_m,
        annotation.dst_SE3_object.rotation)

        detected_bbox = detection.s_pose_ious[frame_id]
        detected_bbox_visuals = detection.s_pose_visuals[frame_id]
        evaluation[track_uuid][frame_id]["iou_3d"] = {}
        evaluation[track_uuid][frame_id]["instance_id"] = {}
        evaluation[track_uuid][frame_id]["translation_error"] = {}
        evaluation[track_uuid][frame_id]["orientation_error"] = {}
        evaluation[track_uuid][frame_id]["iou_3d"] = iou_3d(annotation_bbox.iou, detected_bbox)
        evaluation[track_uuid][frame_id]["instance_id"] = detection.id
        center_annotation = np.array([annotation.dst_SE3_object.translation[0], annotation.dst_SE3_object.translation[1],
        annotation.dst_SE3_object.translation[2]])
        
        center_detected_bbox_visuals = np.array([detected_bbox_visuals.x, detected_bbox_visuals.y, detected_bbox_visuals.z])
        evaluation[track_uuid][frame_id]["translation_error"] = compute_translation_error(center_annotation, center_detected_bbox_visuals)
        
        evaluation[track_uuid][frame_id]["orientation_error"] = compute_orientation_error(annotation.dst_SE3_object.rotation, 
                                                                                           detected_bbox_visuals.rot)

        

    # if frame_id == 1:
    #     break
    
# print("evaluation\n", evaluation)

In [6]:
# https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
df = pd.DataFrame.from_records(
    [
        (track_uuid, frame_id, attributes['iou_3d'], attributes['instance_id']
         , attributes['translation_error'], attributes['orientation_error'])
        for track_uuid, info in evaluation.items()
        for frame_id, attributes in info.items()
    ],
    columns=['track_uuid', 'frame_id', 'iou_3d', 'instance_id', 'translation_error', 'orientation_error']
)
df

,track_uuid,frame_id,iou_3d,instance_id,translation_error,orientation_error
0,1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85,0,0.69647,0,0.279806,6.050542
1,1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85,1,0.68333,0,0.340411,4.924956
2,1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85,2,0.64030,0,0.415170,1.153551
3,1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85,3,0.72589,0,0.186649,0.684752
4,1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85,4,0.67001,0,0.174458,8.624183
...,...,...,...,...,...,...
299,621365dc-b823-40b1-bd85-249166b3ab82,122,0.40151,47,0.518934,1.131774
300,621365dc-b823-40b1-bd85-249166b3ab82,124,0.43274,47,0.452491,7.675689
301,621365dc-b823-40b1-bd85-249166b3ab82,125,0.39079,47,0.911821,5.219381
302,621365dc-b823-40b1-bd85-249166b3ab82,127,0.49269,47,0.538418,1.520937


In [7]:
def get_results(df, track_uuids):
    '''
    Sum all iou_3d and divide by the number of instance_id
    '''
    for track_uuid in track_uuids:
        df_intance_iou_3d = df.loc[df['track_uuid'] == track_uuid, ['instance_id', 'iou_3d', 'translation_error', 'orientation_error']]
        df_intance_iou_3d_mean = df_intance_iou_3d.groupby("instance_id").mean()
        df_intance_iou_3d_count = df_intance_iou_3d.groupby("instance_id").count()
        merge = df_intance_iou_3d_mean.merge(df_intance_iou_3d_count, left_on='instance_id', right_on='instance_id')
        merge = merge.rename(columns={"iou_3d_x": "iou_3d", "translation_error_x": "translation_error",
                                     "orientation_error_x": "orientation_error", "iou_3d_y": "number_of_frames"})
        merge = merge[['iou_3d', 'translation_error', 'orientation_error', 'number_of_frames']]
        print('\n')
        print('Track_UUID', track_uuid)
        print(merge)

In [8]:
def show_interesting_track_uuids(df, minimum_frame_number=30):
    df1 = df.groupby(['track_uuid']).nunique()
    df2 = df1.loc[(df1['frame_id'] > minimum_frame_number), ['frame_id', 'instance_id', 'translation_error', 'orientation_error']]
    return df2

In [9]:
def get_track_uuid(df):
    df3 = df2['frame_id'].keys().to_list()
    return df3

In [10]:
# Important 
df2 = show_interesting_track_uuids(df, 0)
print(df2[['frame_id', 'instance_id']])
track_uuids = get_track_uuid(df)
print(get_results(df, track_uuids))

                                      frame_id  instance_id
track_uuid                                                 
0f4d2bcb-bdee-4c33-b8b8-61258b75e091        62            7
1996a9dc-9ddf-4971-b6cd-95f2f8f7ea85        59            4
621365dc-b823-40b1-bd85-249166b3ab82         5            1
673ff678-82fa-4ac1-aac3-05e330e165e1         9            1
a2b37738-c6aa-43cb-bfcb-aef925680fae        16            3
be9b25df-c8d0-41e7-a338-26196d1080d5        27            7
bf5ace13-42d1-4d38-8b97-9cc7f19bfae9         2            2
c541e364-ff32-4fab-a2e8-3f97f71e06fb        30            4
ecf4920c-3a20-4167-8ec6-4218c4f69248        94            7


Track_UUID 0f4d2bcb-bdee-4c33-b8b8-61258b75e091
               iou_3d  translation_error  orientation_error  number_of_frames
instance_id                                                                  
19           0.669426           0.183515           2.585803                44
36           0.564376           0.339781           1.549

In [11]:
# Tell overall value
# print(df.nunique())
# print(df.head())

# print(df.groupby(['track_uuid']).sum(numeric_only=True))
# print(df.keys())

# print(.keys())
# mylist = np.unique(df2.loc[:, :]['track_uuid'].values).tolist()
# print(df2.loc[:, :].keys())

# print(df.groupby(['track_uuid', 'instance_id']).nunique())
# df.keys() = Index(['track_uuid', 'frame_id', 'iou_3d', 'instance_id'], dtype='object')